In [15]:
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 現在日付の翌日を計算
next_day = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

# Chromeオプションの設定
chrome_options = Options()
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")  # GUIを表示しないモード

driver = webdriver.Chrome(options=chrome_options)

# Tableau User Groupsのイベントページを開く
driver.get("https://usergroups.tableau.com/events/#/list")

# イベントリストが読み込まれるまで待機
wait = WebDriverWait(driver, 60)  # タイムアウト時間を60秒に設定
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "panel-picture-content")))

# イベントが完全に読み込まれるまで待機
time.sleep(10)  # 追加の待機時間を設定

# イベントが読み込まれるのを待つ
events = driver.find_elements(By.CLASS_NAME, "panel-picture-content")

# 最初の5つのイベント情報を抽出
event_details = []
for event in events[:5]:
    try:
        date_element = event.find_element(By.CLASS_NAME, "date")
        date = date_element.text
        
        # タイトル要素の取得
        title_element = event.find_element(By.CLASS_NAME, "general-body--color")
        title = title_element.text
        
        # 詳細リンクの取得
        details_element = event.find_element(By.CSS_SELECTOR, "a.EventRectangle-styles-viewDetails-PsfIW")
        details_link = details_element.get_attribute("href")
        
        # ベースURLと詳細リンクの結合
        base_url = ""
        full_url = base_url + details_link
        
        event_details.append({
            "URL": full_url,
            "完了": 1,
            "コメント": title + ' ' + date,
            "曜日": 0,
            "AMPM": 0,
            "日付": next_day
        })
    except Exception as e:
        print(f"Error extracting event details: {e}")

# ドライバーを閉じる
driver.quit()

# CSVファイルに書き込む
csv_file = "event_details.csv"
csv_columns = ["URL", "完了", "コメント", "曜日", "AMPM", "日付"]

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in event_details:
            writer.writerow(data)
except IOError:
    print("I/O error")

print(f"CSV file '{csv_file}' has been created.")


CSV file 'event_details.csv' has been created.


In [13]:
pip install gspread oauth2client pandas

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 16.8 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/11.5 MB 20.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/11.5 MB 21.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.5 MB 25.4 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/11.5 MB 20.6 MB/s eta 0:00:01
   --------------- ------------------------ 4.6/11.5 MB 20.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.5 MB 20.1 MB/s eta 0:00:01

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Google Sheets APIの認証情報を設定
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('service_account.json', scope)
client = gspread.authorize(creds)

# Google Sheetsの指定されたシートを開く
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1RAZwuhJkpkca4QaVLVO9fzW68TKqLTxIhSfhrGR1Jjw/edit")
sheet = spreadsheet.sheet1

# Google Sheetsのデータを取得
existing_data = sheet.get_all_records()
existing_urls = [row['URL'] for row in existing_data]

# CSVファイルのデータを読み込む
csv_file = "event_details.csv"
df = pd.read_csv(csv_file)

# Google Sheetsに存在しないURLの行を追加
new_rows = []
for _, row in df.iterrows():
    if row['URL'] not in existing_urls:
        new_rows.append(row.tolist())

if new_rows:
    sheet.append_rows(new_rows)

print("Google Sheetsに新しい行が追加されました。")


Google Sheetsに新しい行が追加されました。
